In [2]:
#Import Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from ErcotApi import ercot_connection
import seaborn as sns
pd.options.mode.chained_assignment = None 
#urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


## Real Time Prices

In [2]:
# def ercot_fundamentlas(for_act):
#     erc_historical = pd.read_pickle("erc_fundamentals.pkl")
#     if for_act == "actual":
#         dtfrom = max(erc_historical["Date_y"])
#         dtto = datetime.date.today() + datetime.timedelta(days=0)
#         solar = 159280
#         wind = 110218
#         load = 112403
#     else:
#         dtfrom = datetime.date.today() + datetime.timedelta(days=0)
#         dtto = datetime.date.today() + datetime.timedelta(days=2)
#         solar = 159277
#         wind = 45358
#         load = 2635
    
#     rtm_hub = nrg_clean(str(dtfrom), str(dtto), ["prices", "rtm_north"], [92594])
#     erc_fundamentals = nrg_clean(str(dtfrom), str(dtto), ["fundamentals","solar", "wind", "outages", "load"], [solar, wind, 127656, load])

#     erc_fundies = pd.merge(pd.merge(pd.merge(erc_fundamentals[0], erc_fundamentals[1][["Date","wind"]], on="Date", how="left"), 
#                             erc_fundamentals[2][["Date", "outages"]],on = "Date",how="left"),
#                             erc_fundamentals[3][["Date", "load"]],on = "Date",how="left")     
    
#     erc_fundies["net_load"] = erc_fundies["load"] + erc_fundies["outages"] - erc_fundies["solar"] - erc_fundies["wind"]
#     if for_act == "actual":
#         erc_fundies = pd.merge(erc_fundies, rtm_hub[0][["Date", "rtm_north", "Year", "Month", "Day", "Hour"]], on = ["Year", "Month", "Day", "Hour"],how = "left")
#         if  erc_fundies["Date_y"].unique()[0] not in erc_historical["Date_y"]:
#            all_erc = pd.concat([erc_fundies,erc_historical], ignore_index=True)
#            all_erc.to_pickle("erc_fundamentals.pkl")
#            return(all_erc)
#         else:
#            return(erc_historical) 
#     else:
            
#         erc_fundies["Date_y"] = pd.to_datetime(erc_fundies["Date"], format="%b %d %Y %H:00").dt.date
#         return(erc_fundies)


# erc_funHist = ercot_fundamentlas("actual")
# erc_funFor = ercot_fundamentlas("forecast") 





In [3]:
#Price Simulation
def rtm_simulation(today_fundies, hist_fundies):

    today_peak_nl = max(today_fundies["net_load"])

    befo_historical = hist_fundies[hist_fundies["Date_y"] != dtto]
    unique_nl_filter = befo_historical[(befo_historical["net_load"] >= today_peak_nl-500) & (befo_historical["net_load"] <= today_peak_nl+500)]["Date_y"].unique()

    hist_nl = befo_historical[befo_historical["Date_y"].isin(unique_nl_filter)]
    log_prices = []
    for j in hist_nl["rtm_north"]:
        if j <= 0:
            log_prices.append(0)
        else:
            log_prices.append(np.log(j))

    hist_nl["log_prices"] = log_prices

    sim_prices = pd.DataFrame()
    for i in range(0, 24):
        hourly_prices = hist_nl[hist_nl["Hour"]==i]
        with np.errstate(divide='ignore', invalid='ignore'):
            log_price = np.random.normal(np.mean(hourly_prices["log_prices"]),np.std(hourly_prices["log_prices"]),10000)
        exp_log_price = pd.DataFrame([{'Hour':i,'P10':np.exp(np.percentile(log_price,10)), 'Avg':np.exp(np.median(log_price)), 'P90':np.exp(np.percentile(log_price,90))}])
        sim_prices = pd.concat([sim_prices, exp_log_price], ignore_index=True)

    sim_prices["Hour"] =  sim_prices["Hour"] +1

    deliveryHour = [datetime.time(num-1).strftime("%H:00:00") for num in sim_prices["Hour"]]
    sim_prices["Date"] = str(dtto)
    sim_prices["DateTime"] = pd.to_datetime(sim_prices["Date"] + " "+ deliveryHour, format = "%Y-%m-%d %H:%M:%S")
    
    prices_table = pd.melt(sim_prices, id_vars=["DateTime","Hour"], value_vars=["Avg","P10",  "P90"])
    #viz_table = prices_table.pivot(index = ["variable"], columns="Hour", values="value").reset_index()
    return(sim_prices, prices_table)



In [6]:
#Bal Day
#ERCOT Prices Calls
dtfrom = datetime.date.today() + datetime.timedelta(days=-6) 
dtto = datetime.date.today() + datetime.timedelta(days=-6) 

#hubs_lz = ["HB_NORTH", "HB_HOUSTON", "HB_SOUTH", "HB_WEST", "LZ_NORTH", "LZ_HOUSTON", "LZ_SOUTH", "LZ_WEST"]
hubs_lz = ["HB_NORTH"]

rtm_url = ["np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=HU&size=1000",
           "np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=LZ&size=1000"]

#rtm_url = ["np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=HU&size=1000"]

da_url = ["np4-190-cd/dam_stlmnt_pnt_prices?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPoint=HB_NORTH&size=1000"]


rtm_hublz = ercot_connection(rtm_url)
da_hublz = ercot_connection(da_url)

#ERC Prices Data Clean
def ercot_prices(price_data, market):
    prices = pd.DataFrame()
    for df in price_data:     
        
        if  market.upper() == "RT":
            df =df[df["settlementPoint"]!= "HB_PAN"]
            spp = df
            #spp  = df.groupby(["deliveryDate", "deliveryHour", "settlementPoint"])["settlementPointPrice"].mean().reset_index()
            spp["market"] = "RT"
        else:
            df["market"] = "DA"
            spp = df.rename({"hourEnding":"deliveryHour"}, axis=1)
            spp["deliveryHour"] = pd.to_numeric(spp["deliveryHour"].str.split(":").str[0])
            spp = spp.drop('DSTFlag', axis=1)
            
        deliveryHour = [datetime.time(num-1).strftime("%H:00:00") for num in spp["deliveryHour"]]
        spp["DateTime"] = pd.to_datetime(spp["deliveryDate"] + " "+ deliveryHour, format = "%Y-%m-%d %H:%M:%S")
        spp["type"] = spp["settlementPoint"].str.split("_").str[0]

        prices = pd.concat([prices, spp], ignore_index=True)
    return(prices)
rt_hub_lz = ercot_prices(rtm_hublz, "RT")
#da_hub_lz = ercot_prices(da_hublz, "DA")

# all_da_rt = pd.concat([da_hub_lz, rt_hub_lz], ignore_index=True)
# all_da_rt = all_da_rt.sort_values(by = "DateTime")

# # Current Day Price Plots
# latest_snapshot = datetime.datetime.now()
# today_sim = erc_funFor[erc_funFor["Date_y"]==datetime.date.today()]
# sim_raw, melt_price = rtm_simulation(today_sim, erc_funHist)


# north_prices = all_da_rt[all_da_rt.settlementPoint == "HB_NORTH"]

# price_darts = north_prices.pivot(index = ["deliveryDate", "deliveryHour", "type"], columns="market", values="settlementPointPrice").reset_index()
# price_darts["DART"] = price_darts["DA"] - price_darts["RT"]
# price_d = pd.melt(price_darts, id_vars=["deliveryDate","deliveryHour"], value_vars=["DA","RT",  "DART"])

# north_actual = pd.DataFrame({'DateTime':price_d["deliveryDate"],'Hour':price_d["deliveryHour"],'variable':price_d["market"],
#                            'value':price_d["value"]})
# sim_actual = pd.concat([melt_price, north_actual], ignore_index=True)

# sim_plot = sim_actual[(sim_actual["variable"]=="Avg") | (sim_actual["variable"]== "RT") | (sim_actual["variable"] == "DA")].plot(x = "Hour", y = "value", color= "variable", title= "NORTH HUB Prices: "+str(datetime.date.today())+"|Latest Snapshot:"+str(latest_snapshot))
# sim_plot.show()



# viz_table = sim_actual.pivot(index = ["variable"], columns="Hour", values="value").reset_index().sort_values(by=1, ascending=False)
# viz_table.sort_values(by=1, ascending=False)

# viz_table["Off Peak"] = (viz_table.iloc[:,1:7].mean(axis=1) + viz_table.iloc[:,23:25].mean(axis=1))/2
# viz_table["On Peak"] = viz_table.iloc[:,7:23].mean(axis=1)
# viz_table["HE18-21"] = viz_table.iloc[:,18:22].mean(axis=1)
# viz_table["Max"] = viz_table.iloc[:,1:25].max(axis=1)
# viz_table["ATC"] = viz_table.iloc[:,1:25].mean(axis=1)
# # set styles
# index_col = {'selector': 'td.data.col0',
#          'props': 'background-color: #f5f5f5; color: #000000; text-align: center;'}


# viz_table.style.set_properties(**{'font-size':'10pt', 'padding':'0em 0.4em'})\
#               .background_gradient(cmap=cm, axis=1)\
#               .format(precision=2, thousands=",", decimal=".")\
#               .set_table_styles([index_col])\
#               .hide(axis="index")



In [7]:
rt_hub_lz.to_csv("Jan8_SPP.csv", index=False)

In [5]:
#Prices for Next Day
latest_snapshot = datetime.datetime.now()
sim_date = datetime.date.today() + datetime.timedelta(days=1)
today_sim = erc_funFor[erc_funFor["Date_y"]==sim_date]
sim_raw, melt_price = rtm_simulation(today_sim, erc_funHist)



sim_plot = melt_price.plot(x = "Hour", y = "value", color= "variable", title= "Prices: "+str(sim_date)+"|Latest Snapshot:"+str(latest_snapshot))
sim_plot.show()



viz_table = melt_price.pivot(index = ["variable"], columns="Hour", values="value").reset_index()
viz_table.sort_values(by=1, ascending=False)

viz_table["Off Peak"] = (viz_table.iloc[:,1:7].mean(axis=1) + viz_table.iloc[:,23:25].mean(axis=1))/2
viz_table["On Peak"] = viz_table.iloc[:,7:23].mean(axis=1)
viz_table["HE18-21"] = viz_table.iloc[:,18:22].mean(axis=1)
viz_table["Max"] = viz_table.iloc[:,1:25].max(axis=1)
viz_table["ATC"] = viz_table.iloc[:,1:25].mean(axis=1)
# set styles
index_col = {'selector': 'td.data.col0',
         'props': 'background-color: #f5f5f5; color: #000000; text-align: center;'}


viz_table.style.set_properties(**{'font-size':'10pt', 'padding':'0em 0.4em'})\
              .background_gradient(cmap=cm, axis=1)\
              .format(precision=2, thousands=",", decimal=".")\
              .set_table_styles([index_col])\
              .hide(axis="index")

variable,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Off Peak,On Peak,HE18-21,Max,ATC
Avg,39.00,36.90,34.31,35.26,40.56,46.05,54.85,55.95,46.87,40.94,44.69,53.04,58.81,80.18,127.41,171.58,260.79,296.85,401.00,590.80,260.82,68.67,46.88,36.53,40.19,163.33,387.37,590.80,122.03
P10,6.33,5.74,5.24,4.43,4.67,4.70,4.55,4.53,5.16,5.19,6.21,9.75,12.96,17.23,14.64,15.58,21.95,30.07,57.43,86.55,40.20,13.85,10.49,8.37,7.31,21.62,53.56,86.55,16.49
P90,255.46,230.80,230.00,263.09,334.36,431.87,710.85,708.61,448.78,339.68,322.43,283.48,264.33,393.85,995.17,"1,935.84","2,898.39","2,914.98","2,757.91","4,178.45","1,706.52",338.61,207.58,164.21,238.41,"1,324.87","2,889.47","4,178.45",971.47


In [6]:
#Prices Summary
#ERCOT Prices Calls for Hubs
def ercot_price_summary(from_days, to_days):
    dtfrom = datetime.date.today() + datetime.timedelta(days=from_days) 
    dtto = datetime.date.today() + datetime.timedelta(days=to_days) 

    rtm_url = ["np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=HU&size=15000"]

    #rtm_url = ["np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=HU&size=1000"]

    da_url = ["np4-190-cd/dam_stlmnt_pnt_prices?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPoint=HB_NORTH&size=10000",
            "np4-190-cd/dam_stlmnt_pnt_prices?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPoint=HB_HOUSTON&size=10000",
            "np4-190-cd/dam_stlmnt_pnt_prices?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPoint=HB_SOUTH&size=10000",
            "np4-190-cd/dam_stlmnt_pnt_prices?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPoint=HB_WEST&size=10000"]



    rt_hub_lz = ercot_prices(ercot_connection(rtm_url), "RT")
    da_hub_lz = ercot_prices(ercot_connection(da_url), "DA")
    realtime_hub = rt_hub_lz.pivot(index=["deliveryDate", "deliveryHour"], columns="settlementPoint", values="settlementPointPrice").rename_axis(None, axis=1).reset_index()
    dayahead_hub = da_hub_lz.pivot(index=["deliveryDate", "deliveryHour"], columns="settlementPoint", values="settlementPointPrice").rename_axis(None, axis=1).reset_index()
    realtime_hub = realtime_hub.rename({'deliveryHour':'Hour'}, axis=1)
    dayahead_hub = dayahead_hub.rename({'deliveryHour':'Hour'}, axis=1)
    dart_spreads = dayahead_hub.iloc[:,2:6] - realtime_hub.iloc[:,2:6]  
    dart_spreads["Date"] = realtime_hub["deliveryDate"]
    dart_spreads["Hour"] = realtime_hub["Hour"]
    #return(realtime_hub, dayahead_hub)
    return(realtime_hub, dayahead_hub, dart_spreads)

#calculatting average for days > 0
def group_means(real_time, day_ahead, day_real,days):
        
    dtfilter = datetime.date.today() + datetime.timedelta(days=-days) 
    realtm_hub = real_time[real_time["deliveryDate"] >= str(dtfilter)].groupby("Hour")[["HB_NORTH","HB_HOUSTON", "HB_SOUTH", "HB_WEST"]].mean().round(2).reset_index()
    dayahd_hub = day_ahead[day_ahead["deliveryDate"] >= str(dtfilter)].groupby("Hour")[["HB_NORTH","HB_HOUSTON", "HB_SOUTH", "HB_WEST"]].mean().round(2).reset_index()
    dt_spreads = day_real[day_real["Date"] >= str(dtfilter)].groupby("Hour")[["HB_NORTH","HB_HOUSTON", "HB_SOUTH", "HB_WEST"]].mean().round(2).reset_index() 
    return(realtm_hub, dayahd_hub, dt_spreads)

#Table formating function
def fig_tables(rt,da,dart):
    fig = make_subplots(rows=1, cols=3,
                        specs=[[{'type':'table'},{'type':'table'},{'type':'table'}]], 
                        horizontal_spacing=0.05,
                        subplot_titles=("Day Ahead", "Real Time", "DART Spreads"))

    fig.add_trace(go.Table(header=dict(values=list(da.columns)), 
                        cells = dict(values= da.transpose().values.tolist())),
                        row=1, 
                        col=1)
    fig.add_trace(go.Table(header=dict(values=list(rt.columns)), 
                        cells = dict(values= rt.transpose().values.tolist())), 
                        row=1,
                        col=2)
    fig.add_trace(go.Table(header=dict(values=list(dart.columns)), 
                        cells = dict(values= dart.transpose().values.tolist())), 
                        row=1,
                        col=3)

    fig.update_layout(
        width = 1500,
        height = 570,
        margin = dict(l=20,r=20, t=50,b=10),
    
    )
    fig.update_annotations(xshift=-170)
    return(fig)


realtime, dayahed, darts = ercot_price_summary(-30, -1)


## Current Day : 

In [10]:

price_dates = datetime.datetime.today() + datetime.timedelta(days=0)
date_today = datetime.datetime.today()
date_delta = price_dates - date_today
realtime_0, dayahed_0, darts_0 = ercot_price_summary(date_delta.days, date_delta.days)
rt_0, da_0, dart_0 = group_means(realtime_0, dayahed_0, darts_0,-date_delta.days)

fig_tables(rt_0, da_0, dart_0).update_layout(title= "Prices for " + str(price_dates.date())).show()

## 10 Day Average

In [8]:

rt_10, da_10, dart_10 = group_means(realtime, dayahed, darts,10)
fig_tables(rt_10, da_10, dart_10).show()

## 30 day Average

In [9]:

rt_30, da_30, dart_30 = group_means(realtime, dayahed, darts,30)
fig_tables(rt_30, da_30, dart_30).show()